In [1]:
import math

In [9]:
def phi(x):
    return 1/2 * (1 + math.erf(x / math.sqrt(2)))

In [10]:
def GBlackScholes(side, S, X, T, r, sigma, q):
    s_side = side.lower()
    assert s_side in ["put", "call"]
    b = r - q
    d1 = (math.log(S/X) + (b + (sigma**2)/2)*T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    ii = 1 if s_side == "call" else -1
    out = ii*S*math.exp((b-r)*T) * phi(ii*d1) - ii*X*math.exp(-r*T) * phi(ii*d2)
    return out
    

In [11]:
def iter(v1,v2,p1,p2,p,vf,pf,epsilon=1e-10):
    vi = vf(v1,v2,p1,p2)
    pv = pf(vi)
    ee = abs(p-pv) < epsilon
    if pv < p:
        vv1 = vi
        pp1 = pv
        vv2 = v2
        pp2 = p2
        return vv1,vv2,pp1,pp2,ee,True
    else:
        vv1 = v1
        pp1 = p1
        vv2 = vi
        pp2 = pv
        return vv1, vv2, pp1, pp2, ee, False

In [17]:
def ImpliedVolatility(side, p, S, X, T, r, q, epsilon=1e-8):
    sside = side.lower()
    assert sside in ["put", "call"]
    if sside == "put":
        assert X <= S
    else:
        assert X >= S
    
    v1_init = 0.005
    v2_init = 4
    
    def vi_func(v1,v2,p1,p2):
        return v1 + (p-p1)*(v2-v1)/(p2-p1)
    def price_func(v):
        return GBlackScholes(sside, S, X, T, r, v, q)
    
    p1_init = price_func(v1_init)
    p2_init = price_func(v2_init)
    
    v1,v2,p1,p2, _, low = iter(v1_init, v2_init, p1_init, p2_init, p, vi_func, price_func, epsilon)
    
    i=0
    while i<1000:
        v1,v2,p1,p2,retBool,low = iter(v1,v2,p1,p2,p,vi_func, price_func)
        if retBool:
            if low:
                return v1
            else:
                return v2
        i += 1
    if low:
        return v1
    else:
        return v2

In [18]:
ImpliedVolatility("call", 6.00, 100, 105, 30/365, .02, 0.0)

0.7003604332868295

In [19]:
ImpliedVolatility("put", 6.00, 100, 95, 30/365, .02, 0.0)

0.7487684002427318

In [20]:
# https://arxiv.org/pdf/1908.02347.pdf
def put_expression(k, s, a):
    return (-1)**(1-a) * s**(-a) * ((a -1)*k + s) - (k - s)**(1 - a)

In [21]:
put_expression(50, 100, 2.75)

(-0.00033285787283606817-0.00033285787283606833j)

In [28]:
# K1 --> closer to the money option with known price p_K1 / c_K1
# K2 --> out-of-the-money option we are trying to price relatively
# S0 --> underlying price
# alpha --> tail distribution exponent

In [53]:
def relative_put(K1, p_K1, K2, S0, alpha=2.75):
    assert K1 > K2
    ee = p_K1 * put_expression(K2, S0, alpha) / put_expression(K1, S0, alpha)
    return ee.real

In [67]:
p_K2 = relative_put(290, 2.90, 200, 317.89, 2.75)# <--- prices from 290 SPY strike w/30 days left at 3:40pm EST 7/14/20
p_K2 #<--- damn close; 200 strike is $.14 bid / $.15 offer

0.15214072133580922

In [83]:
ImpliedVolatility("put", 2.90,317.89,290, 31/365, 0.0, 0.0)

0.3411224959974112

In [84]:
ImpliedVolatility("put", p_K2,317.89,200, 31/365, 0.0, 0.0)

0.6742495741089689

In [61]:
def relative_call_2(K1, c_K1, K2, S0, alpha=2.75):
    assert K1 < K2
    return c_K1 * ((K2-S0)/(K1-S0))**(1-alpha)

In [87]:
c_K2 = relative_call_2(325, 5.30, 350, 317.90, 2.75) # prices from SPY 320 strike 3:45 EST 7/14/20
c_K2 # <-- again damn close to market b/o at $.35/$.36

0.3780891138970195

In [88]:
ImpliedVolatility("call", 5.30,317.90,325, 31/365, 0.0, 0.0)

0.22415623235315038

In [89]:
ImpliedVolatility("call", c_K2,317.90,350, 31/365, 0.0, 0.0)

0.1976866639825468